# CNN 1D multimodal (IMU + Plantar)


In [2]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("CUDA available:", torch.cuda.is_available())
print("Device:", device)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
Device: cuda
GPU: NVIDIA RTX 6000 Ada Generation


In [4]:
def smart_read_csv(path):
    for sep in [";", "\t", ","]:
        try:
            df = pd.read_csv(path, sep=sep)
            if df.shape[1] > 1:
                return df
        except Exception:
            pass
    raise ValueError(f"Impossible de lire {path}")


def find_time_col(df):
    for c in df.columns:
        if "time" in c.lower():
            return c
    raise ValueError("Colonne temps non trouvee")


def read_and_slice_by_time(path, t0, t1):
    df = smart_read_csv(path)
    tcol = find_time_col(df)

    df = df[(df[tcol] >= t0) & (df[tcol] <= t1)]
    df = df.drop(columns=[tcol])

    return torch.tensor(df.values, dtype=torch.float32)


def resample_to_L(x, L):
    """
    x : [T, C] -> [L, C]
    """
    T, C = x.shape

    if T == 0:
        return torch.zeros(L, C)

    if T == L:
        return x

    idx = torch.linspace(0, T - 1, L)
    idx0 = idx.long()
    idx1 = torch.clamp(idx0 + 1, max=T - 1)
    w = idx - idx0

    return (1 - w).unsqueeze(1) * x[idx0] + w.unsqueeze(1) * x[idx1]


In [5]:
ROOTS = {
    "imu": "/home/fisa/stockage1/mindscan/IMU",
    "plantar": "/home/fisa/stockage1/mindscan/Plantar_activity",
    "events": "/home/fisa/stockage1/mindscan/Events",
}

FILENAMES = {
    "imu": "imu.csv",
    "plantar": "insoles.csv",
}

L = 256
NUM_CLASSES = 31


In [6]:
def build_segments_index(events_root):
    rows = []
    subjects = sorted([
        d for d in os.listdir(events_root)
        if os.path.isdir(os.path.join(events_root, d))
    ])

    for subject in subjects:
        for seq in os.listdir(os.path.join(events_root, subject)):
            classif = os.path.join(events_root, subject, seq, "classif.csv")
            if not os.path.exists(classif):
                continue

            df = pd.read_csv(classif, sep=";")
            for _, r in df.iterrows():
                rows.append({
                    "subject": subject,
                    "seq": seq,
                    "label": int(r["Class"]),
                    "t0": float(r["Timestamp Start"]),
                    "t1": float(r["Timestamp End"]),
                })

    return pd.DataFrame(rows)


segments = build_segments_index(ROOTS["events"])
print("Total segments:", len(segments))


Total segments: 10204


In [7]:
allowed_subjects = {f"S{str(i).zfill(2)}" for i in range(1, 25)}
segments = segments[segments["subject"].isin(allowed_subjects)].reset_index(drop=True)

subjects = sorted(segments["subject"].unique())
np.random.seed(0)
np.random.shuffle(subjects)

n_train = int(0.8 * len(subjects))
train_subjects = set(subjects[:n_train])
val_subjects = set(subjects[n_train:])

train_segments = segments[segments["subject"].isin(train_subjects)].reset_index(drop=True)
val_segments = segments[segments["subject"].isin(val_subjects)].reset_index(drop=True)

print("Train segments:", len(train_segments))
print("Val segments:", len(val_segments))


Train segments: 6084
Val segments: 1569


In [8]:
class MultiModalEventDataset(Dataset):
    def __init__(self, segments_df, roots, filenames, L=256):
        self.df = segments_df.reset_index(drop=True)
        self.roots = roots
        self.filenames = filenames
        self.L = L

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        r = self.df.iloc[i]
        subject, seq = r["subject"], r["seq"]
        t0, t1 = float(r["t0"]), float(r["t1"])
        y = int(r["label"]) - 1

        imu_path = os.path.join(self.roots["imu"], subject, seq, self.filenames["imu"])
        plantar_path = os.path.join(self.roots["plantar"], subject, seq, self.filenames["plantar"])

        imu = read_and_slice_by_time(imu_path, t0, t1)            # [T, 96]
        imu = resample_to_L(imu, self.L).transpose(0, 1).contiguous()  # [96, 256]

        plantar = read_and_slice_by_time(plantar_path, t0, t1)    # [T, C]
        plantar = resample_to_L(plantar, self.L).transpose(0, 1).contiguous()  # [C, 256]

        return {
            "imu": imu,
            "plantar": plantar,
            "y": torch.tensor(y, dtype=torch.long),
        }


In [9]:
train_ds = MultiModalEventDataset(train_segments, ROOTS, FILENAMES, L)
val_ds = MultiModalEventDataset(val_segments, ROOTS, FILENAMES, L)

train_dl = DataLoader(
    train_ds,
    batch_size=128,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
)

val_dl = DataLoader(
    val_ds,
    batch_size=128,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
)

b = next(iter(train_dl))
imu_channels = b["imu"].shape[1]
plantar_channels = b["plantar"].shape[1]
print("Train batch IMU:", b["imu"].shape)
print("Train batch Plantar:", b["plantar"].shape)
print("Train batch y:", b["y"].shape)
print("Channels -> IMU:", imu_channels, "| Plantar:", plantar_channels)


Train batch IMU: torch.Size([128, 96, 256])
Train batch Plantar: torch.Size([128, 50, 256])
Train batch y: torch.Size([128])
Channels -> IMU: 96 | Plantar: 50


In [10]:
class ConvBranch(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(in_channels, 64, 7, stride=2, padding=3),
            nn.ReLU(),
            nn.Conv1d(64, 128, 5, stride=2, padding=2),
            nn.ReLU(),
            nn.Conv1d(128, 256, 3, stride=2, padding=1),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.features(x)
        return x.mean(dim=-1)  # [B, 256]


class IMUPlantarMultiCNN(nn.Module):
    def __init__(self, imu_in_channels=96, plantar_in_channels=32, num_classes=31):
        super().__init__()
        self.imu_branch = ConvBranch(imu_in_channels)
        self.plantar_branch = ConvBranch(plantar_in_channels)

        # Fusion multilayer des deux representations [256 + 256]
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
        )

    def forward(self, imu_x, plantar_x):
        imu_feat = self.imu_branch(imu_x)
        plantar_feat = self.plantar_branch(plantar_x)
        fused = torch.cat([imu_feat, plantar_feat], dim=1)
        return self.classifier(fused)


In [11]:
def train_multimodal(model, train_dl, val_dl, epochs=20, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = total = 0

        pbar = tqdm(train_dl, desc=f"Epoch {epoch + 1}/{epochs}")
        for batch in pbar:
            imu_x = batch["imu"].to(device, non_blocking=True)
            plantar_x = batch["plantar"].to(device, non_blocking=True)
            y = batch["y"].to(device, non_blocking=True)

            optimizer.zero_grad()
            logits = model(imu_x, plantar_x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imu_x.size(0)
            correct += (logits.argmax(1) == y).sum().item()
            total += imu_x.size(0)

            pbar.set_postfix(
                loss=f"{running_loss / total:.4f}",
                acc=f"{correct / total:.3f}",
            )

        model.eval()
        val_correct = val_total = 0
        with torch.no_grad():
            for batch in val_dl:
                imu_x = batch["imu"].to(device)
                plantar_x = batch["plantar"].to(device)
                y = batch["y"].to(device)
                logits = model(imu_x, plantar_x)
                val_correct += (logits.argmax(1) == y).sum().item()
                val_total += y.size(0)

        print(f"Val acc: {val_correct / val_total:.3f}")


In [ ]:
model = IMUPlantarMultiCNN(
    imu_in_channels=imu_channels,
    plantar_in_channels=plantar_channels,
    num_classes=NUM_CLASSES,
).to(device)

train_multimodal(model, train_dl, val_dl, epochs=5, lr=1e-3)


Epoch 1/5: 100%|██████████| 48/48 [04:23<00:00,  5.48s/it, acc=0.290, loss=2.4598] 


Val acc: 0.549


Epoch 2/5:  79%|███████▉  | 38/48 [02:46<00:30,  3.01s/it, acc=0.632, loss=1.1416]

In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            imu_x = batch["imu"].to(device)
            plantar_x = batch["plantar"].to(device)
            y = batch["y"].to(device)

            logits = model(imu_x, plantar_x)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


val_acc = evaluate(model, val_dl)
print(f"Validation accuracy (IMU + Plantar): {val_acc:.3f}")


NameError: name 'model' is not defined

In [ ]:
sample = val_ds[0]
imu_x = sample["imu"].unsqueeze(0).to(device)
plantar_x = sample["plantar"].unsqueeze(0).to(device)
y_true = sample["y"].item()

model.eval()
with torch.no_grad():
    logits = model(imu_x, plantar_x)
    y_pred = logits.argmax(dim=1).item()

print("Vrai label :", y_true)
print("Label predit :", y_pred)
